In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
import datetime
import calendar

In [ ]:
dir = 'drive/MyDrive/3001 Project/data/'

cal = pd.read_csv(f'{dir}raw data/calendar.csv')
sell = pd.read_csv(f'{dir}raw data/sell_prices.csv.zip', compression = 'zip')
train_eval = pd.read_csv(f'{dir}raw data/sales_train_evaluation.csv.zip',\
                         compression = 'zip')


# Visualization

In [ ]:
def plot_sample_sale(sales, c, label):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x = list(range(sales.size)), y = sales,\
                           mode = 'lines', marker = dict(color = c)))

  fig.update_layout(height = 400, width = 800, title_text = label,\
                  xaxis_title = 'Days', yaxis_title = 'Numer of Units Sold')

  fig.show()

In [ ]:
idx = 1
label = 'Sale of ID: ' + train_val.loc[idx, ['id']].values[0] + ' (full)'
plot_sample_sale(train_val.iloc[idx, 6:], 'green', label)

In [ ]:
label = 'Sale of ID: ' + train_val.loc[idx, ['id']].values[0] + ' (Day 1 ~ 200)'
plot_sample_sale(train_val.iloc[idx, 6:206], 'green', label)

In [ ]:
label = 'Sale of ID: ' + train_val.loc[idx, ['id']].values[0] + ' (Day 600 ~ 800)'
plot_sample_sale(train_val.iloc[idx, 605:805], 'green', label)

In [ ]:
label = 'Sale of ID: ' + train_val.loc[idx, ['id']].values[0] + ' (Day 1650 ~ 1850)'
plot_sample_sale(train_val.iloc[idx, 1655:1855], 'green', label)

In [ ]:
sales_df = train_val.set_index('id').iloc[:, 5:].T.merge(cal.set_index('d')['date'],\
                                                         left_index = True,\
                                                         right_index = True,\
                                                         validate = '1:1').set_index('date')

fig = go.Figure()
for s in sell['store_id'].unique():
  items = [i for i in sales_df.columns if s in i]
  rolling_mean = sales_df[items].sum(axis = 1).rolling(28).mean()
  fig.add_trace(go.Scatter(x = list(range(rolling_mean.size)), y = rolling_mean,\
                           name = s))

fig.update_layout(yaxis_title = 'Mean Sales', xaxis_title = 'Time',\
                  title = 'Rolling Mean Sales (28 Days) v.s. Time for Stores')

fig.show()


In [ ]:
fig = go.Figure()

for i,store in enumerate(sell['store_id'].unique()):
  items = [col for col in sales_df.columns if store in col]
  rolling_mean = sales_df[items].sum(axis = 1).rolling(28).mean()
  fig.add_trace(go.Box(x = [store]*rolling_mean.size, y = rolling_mean,\
                       name = store))
  
fig.update_layout(yaxis_title = 'Rolling Mean Sales', xaxis_title = 'Stores',\
                  title = 'Rolling Mean Sales (28 Days) for Stores')


In [ ]:

fig = go.Figure()

for i,store in enumerate(train_eval['cat_id'].unique()):
  items = [col for col in sales_df.columns if store in col]
  rolling_mean = sales_df[items].sum(axis = 1).rolling(28).mean()
  fig.add_trace(go.Box(x = [store]*rolling_mean.size, y = rolling_mean,\
                       name = store))
  
fig.update_layout(yaxis_title = 'Rolling Mean Sales', xaxis_title = 'Categories',\
                  title = 'Rolling Mean Sales (28 Days) for Categories')

In [ ]:
item_cnt = train_val.groupby(['store_id', 'cat_id']).agg({'item_id': 'nunique'}).reset_index()
item_cnt.rename(columns = {'item_id': 'item_cnt'}, inplace = True)
item_cnt.head()

,store_id,cat_id,item_cnt
0,CA_1,FOODS,1437
1,CA_1,HOBBIES,565
2,CA_1,HOUSEHOLD,1047
3,CA_2,FOODS,1437
4,CA_2,HOBBIES,565


In [ ]:
fig = px.bar(item_cnt, x = 'store_id', y = 'item_cnt',\
             color = 'cat_id', title = 'Item Counts for Each Store and Category')
fig.show()

In [ ]:
tmp = train_eval[train_eval['cat_id'] == 'FOODS'][['dept_id',\
                                                   'store_id']].groupby(['dept_id',\
                                                                         'store_id']).size().reset_index()

tmp.columns = ['dept_id', 'store_id', 'product_cnt']
tmp.head()

,dept_id,store_id,product_cnt
0,FOODS_1,CA_1,216
1,FOODS_1,CA_2,216
2,FOODS_1,CA_3,216
3,FOODS_1,CA_4,216
4,FOODS_1,TX_1,216


In [ ]:
fig = px.bar(tmp, x = 'store_id', y = 'product_cnt',\
             color = 'dept_id', title = 'Item Counts for Each Store and Category in FOODS')
fig.show()

In [ ]:
sub_cal = cal[['weekday', 'd']]
sub_cal.set_index('d', inplace = True)
weekday_dict = {'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3,\
                'Thursday': 4, 'Friday': 5, 'Saturday': 6}

def plot_weekday_avg(state):
  df = train_val.set_index('id')
  df = df[df.state_id == state]
  df = df.iloc[:, 5:].T
  df = df.sum(axis = 1).to_frame()
  df.columns = ['purchase_cnt']
  df = sub_cal.merge(df, left_index = True, right_index = True)
  weekday_cnt = df.groupby(['weekday']).size().to_frame()
  weekday_cnt.columns = ['weekday_cnt']
  purchase_sum = df.groupby(['weekday']).sum()
  df = weekday_cnt.merge(purchase_sum, left_index = True, right_index = True)
  df['purchase_avg'] = df['purchase_cnt']/df['weekday_cnt']
  df = df.reset_index()
  df['weekday_num'] = [weekday_dict[day] for day in df['weekday']]

  return df.sort_values(by = ['weekday_num'])

In [ ]:
CA_df = plot_weekday_avg('CA')

fig = px.bar(CA_df, x = 'weekday', y = 'purchase_avg', title = 'Averaged Purchase Counts v.s. Weekdays for Stores in CA')
fig.show()


In [ ]:
TX_df = plot_weekday_avg('TX')
fig = px.bar(TX_df, x = 'weekday', y = 'purchase_avg', title = 'Averaged Purchase Counts v.s. Weekdays for Stores in TX')
fig.show()

In [ ]:
WI_df = plot_weekday_avg('WI')
fig = px.bar(WI_df, x = 'weekday', y = 'purchase_avg',\
             title = 'Averaged Purchase Counts v.s. Weekdays for Stores in WI')
fig.show()

In [ ]:

def plot_snap(state):
  col = 'snap_'+state
  snap_cal = cal[cal[col] == 1]
  non_snap_cal = cal[cal[col] == 0]
  trans_train = train_val[(train_val.state_id == state)]
  trans_train.set_index('id', inplace = True)
  trans_train = trans_train.iloc[:, 6:].T
  trans_train = trans_train.sum(axis = 1).to_frame()
  trans_train.columns = ['sales_cnt']
  snap_train = trans_train[trans_train.index.isin(snap_cal['d'])]
  non_snap_train = trans_train[trans_train.index.isin(non_snap_cal['d'])]
  plot_df = pd.DataFrame()
  plot_df['snap'] = ['SNAP', 'No_SNAP']
  plot_df['sales_mean'] = [0, 0]
  plot_df.iloc[0, 1] = snap_train.mean().values
  plot_df.iloc[1, 1] = non_snap_train.mean().values

  fig = px.pie(plot_df, values = 'sales_mean', names = 'snap',\
               title = 'Sales for SNAP v.s. no SNAP in '+state)
  fig.show()


In [ ]:
plot_snap('CA')

In [ ]:
plot_snap('TX')

In [ ]:
plot_snap('WI')

# Feature Engineering

In [ ]:
# Days with 2 events - 2 events are not the same category, so we'll just take max
tmp = pd.get_dummies(cal, prefix = ['type1', 'type2'],\
                     columns = ['event_type_1', 'event_type_2'])

tmp['Cultural'] = tmp[["type1_Cultural", "type2_Cultural"]].max(axis = 1)
tmp.rename(columns = {'type1_National': 'National'}, inplace = True)
tmp['Religious'] = tmp[["type1_Religious", "type2_Religious"]].max(axis = 1)
tmp.rename(columns = {'type1_Sporting': 'Sporting'}, inplace = True)

tmp = tmp.drop(['type1_Cultural', 'type1_Religious', 'type2_Cultural',\
                'type2_Religious'], axis = 1)
tmp['event'] = tmp[['National', 'Sporting', 'Cultural',\
                    'Religious']].max(axis = 1)

tmp = tmp.drop(['event_name_1', 'event_name_2', 'date', 'wm_yr_wk', 'weekday',\
                'year', 'National', 'Sporting', 'Cultural', 'Religious'], axis = 1)
     
tmp

,wday,month,d,snap_CA,snap_TX,snap_WI,event
0,1,1,d_1,0,0,0,0
1,2,1,d_2,0,0,0,0
2,3,1,d_3,0,0,0,0
3,4,2,d_4,1,1,0,0
4,5,2,d_5,1,0,1,0
...,...,...,...,...,...,...,...
1964,5,6,d_1965,0,1,1,0
1965,6,6,d_1966,0,0,0,0
1966,7,6,d_1967,0,0,0,0
1967,1,6,d_1968,0,0,0,0


In [ ]:
def change_season(today):
  """
  This function checks if given date is at the first or last week of the current season.
  @ Output: (int) 1 if the date is at the first or last week of the current season.
                  0 otherwise
  """
  if today.month in [1, 4, 7, 10]:
    if today.day in calendar.monthcalendar(today.year, today.month)[0]:
      return 1
    else:
      return 0
  elif today.month in [3, 6, 9, 12]:
    if today.day in calendar.monthcalendar(today.year, today.month)[-1]:
      return 1
    else:
      return 0
  
  return 0


In [ ]:
tmp['date'] = pd.to_datetime(tmp['date'], infer_datetime_format = True)
tmp['change_season'] = [int(change_season(d)) for d in tmp['date']]

In [ ]:
tmp.to_csv('drive/MyDrive/3001 Project/data/calendar_w_dummies.csv', index = False)

In [ ]:
tmp.drop(['event_name_1', 'event_name_2', 'weekday'], axis = 1).head()

,date,wm_yr_wk,wday,month,year,d,snap_CA,snap_TX,snap_WI,National,Sporting,Cultural,Religious,spring,summer,fall,change_season,weekend
0,2011-01-29,11101,1,1,2011,d_1,0,0,0,0,0,0,0,0,0,0,0,1
1,2011-01-30,11101,2,1,2011,d_2,0,0,0,0,0,0,0,0,0,0,0,1
2,2011-01-31,11101,3,1,2011,d_3,0,0,0,0,0,0,0,0,0,0,0,0
3,2011-02-01,11101,4,2,2011,d_4,1,1,0,0,0,0,0,0,0,0,0,0
4,2011-02-02,11101,5,2,2011,d_5,1,0,1,0,0,0,0,0,0,0,0,0
